<h1 align=center>Capstone Project</h1>

<h2>Data Gathering</h2>

In [1]:
#Importing packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
import json # library to handle JSON files

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [48]:
#Enter Foursquare credentials
CLIENT_ID = 'Enter your foursquare id' # your Foursquare ID
CLIENT_SECRET = 'Enter your Foursquare secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Enter your foursquare id
CLIENT_SECRET:Enter your Foursquare secret


In [3]:
#install beautifulsoup for webscraping
!pip install beautifulsoup4

In [4]:
#Gathering data from the website and storing in th bangalore.csv file
from bs4 import BeautifulSoup
url="https://www.indiatvnews.com/pincode/karnataka/bangalore/bangalore-city"
html = requests.get(url).content
df_list = pd.read_html(html,flavor='bs4')
df=df_list[1]
df.to_csv('bangalore.csv')
df.head()

,Office,Taluk,District,State,Pincode
0,A F Station Yelahanka,Bangalore North,Bangalore,KARNATAKA,560063
1,Agram,Bangalore South,Bangalore,KARNATAKA,560007
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024


In [5]:
#Method to get coordinates of the localities 
def coords(off,dis):
    address=off+','+dis
    geolocator = Nominatim(user_agent="ny_explorer")
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    except:
        return(np.nan,np.nan)
    return(latitude, longitude)

In [6]:
#Adding the Latitude and Longitude columns to the dataframe
long=list()
lat=list()
for office,district in zip(df['Office'],df['District']):
    la,lo=coords(office,district)
    lat.append(la)
    long.append(lo)
df['Latitude']=lat
df['Longitude']=long
df.head()

,Office,Taluk,District,State,Pincode,Latitude,Longitude
0,A F Station Yelahanka,Bangalore North,Bangalore,KARNATAKA,560063,NaN,NaN
1,Agram,Bangalore South,Bangalore,KARNATAKA,560007,NaN,NaN
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007,12.964027,77.627500
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024,13.033377,77.589523


<h2>Data Cleaning</h2>

In [7]:
#To check duplicate datas and null values
df.Latitude.value_counts()

12.979120    4
12.971889    2
13.133436    2
12.877349    2
13.016341    2
13.046453    2
13.292399    2
12.800444    2
13.335238    2
12.929273    2
13.000509    2
13.027331    1
13.215192    1
12.962321    1
13.000405    1
12.754299    1
12.928367    1
12.968003    1
13.031462    1
12.915221    1
12.906985    1
13.053333    1
12.860087    1
13.089139    1
12.897971    1
12.968444    1
12.985852    1
12.956997    1
12.886018    1
12.912192    1
12.877710    1
13.010868    1
12.742629    1
13.033117    1
12.945401    1
12.909558    1
12.979745    1
12.936265    1
13.033019    1
12.935146    1
12.895583    1
12.818896    1
13.002655    1
13.012022    1
12.986475    1
12.950624    1
12.856532    1
12.969637    1
13.204259    1
12.971777    1
12.961274    1
12.958577    1
13.171580    1
13.047950    1
12.977673    1
12.964027    1
13.025800    1
12.858081    1
13.296413    1
12.915113    1
12.954345    1
13.056649    1
13.176735    1
13.030013    1
12.976395    1
13.427934    1
12.990879 

In [8]:
#Removing null values
df_new= df[pd.notnull(df['Latitude'])]
df_new

,Office,Taluk,District,State,Pincode,Latitude,Longitude
2,Air Force Hospital,Bangalore North,Bangalore,KARNATAKA,560007,12.964027,77.627500
3,Amruthahalli,Bangalore North,Bangalore,KARNATAKA,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore North,Bangalore,KARNATAKA,560024,13.033377,77.589523
5,Arabic College,Bangalore North,Bangalore,KARNATAKA,560045,13.030013,77.620872
6,Attur,Bangalore North,Bangalore,KARNATAKA,560064,13.107470,77.565685
7,Austin Town,Bangalore North,Bangalore,KARNATAKA,560047,12.961274,77.615294
8,Banaswadi,Bangalore North,Bangalore,KARNATAKA,560043,13.014162,77.651854
10,Bangalore International Airport,NaN,Bangalore,KARNATAKA,560300,13.204259,77.706567
12,Bellandur,Bangalore South,Bangalore,KARNATAKA,560103,12.979120,77.591300
13,Benson Town,Bangalore North,Bangalore,KARNATAKA,560046,12.997803,77.604175


In [9]:
# drop unnecessary columns
column_to_drop = ['Taluk','State']
df_new.drop(columns=column_to_drop, axis=1,inplace=True)
df_new

C:\Users\vishu\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Office,District,Pincode,Latitude,Longitude
2,Air Force Hospital,Bangalore,560007,12.964027,77.627500
3,Amruthahalli,Bangalore,560092,13.066513,77.596624
4,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
5,Arabic College,Bangalore,560045,13.030013,77.620872
6,Attur,Bangalore,560064,13.107470,77.565685
7,Austin Town,Bangalore,560047,12.961274,77.615294
8,Banaswadi,Bangalore,560043,13.014162,77.651854
10,Bangalore International Airport,Bangalore,560300,13.204259,77.706567
12,Bellandur,Bangalore,560103,12.979120,77.591300
13,Benson Town,Bangalore,560046,12.997803,77.604175


In [10]:
#resetting the index
df_new.reset_index(inplace=True,drop=True)
df_new.head()

,Office,District,Pincode,Latitude,Longitude
0,Air Force Hospital,Bangalore,560007,12.964027,77.627500
1,Amruthahalli,Bangalore,560092,13.066513,77.596624
2,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
3,Arabic College,Bangalore,560045,13.030013,77.620872
4,Attur,Bangalore,560064,13.107470,77.565685


In [11]:
# for combining neighbourhood of a single postal code(location) i.e removing duplicate datas and combining into one
l=list()
for a in df_new['Latitude'].unique():
    ne=""
    df_post = df_new[df_new['Latitude']==a].reset_index(drop=True)
    neighs = df_post.loc[:,'Office']
    for n in neighs:
        ne=ne+','+n
    df_post1 = df_post.iloc[0]
    df_post1['Office']=ne[1:]
    l.append(df_post1)
dff = pd.DataFrame(l)
dff.reset_index(drop=True,inplace=True)

C:\Users\vishu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
#Renaming the column 'Office' as 'Locality' which is much suited
dff.columns = ['Locality']+list(map(lambda x: x , dff.columns[1:]))
dff.columns

Index(['Locality', 'District', 'Pincode', 'Latitude', 'Longitude'], dtype='object')

In [13]:
#dff.to_csv('Bangalore_cleaned.csv')

In [14]:
#Visualizing the data points on the map
#from this we realise locations far from the bangalore centre are also present which needs to be removed
latitude=12.9180502
longitude=77.5922633618582
map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, location in zip(dff['Latitude'], dff['Longitude'], dff['District'], dff['Locality']):
    label = '{},{}'.format(location,district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bang)  
    
map_bang

In [15]:
#installing haversine which is used to calculate distance between the coords
!pip install haversine

In [16]:
#removing areas which are 35 kms away from bangalore center so as to make the data more concentrated towards core bangalore.
from haversine import haversine, Unit
bang = coords('bangalore','bangalore')
l=list()
for lat,long in zip(dff['Latitude'], dff['Longitude']):
    if haversine((lat,long),bang)<35:
        df_post1 = dff[dff['Latitude']==lat]
        l.append(df_post1.iloc[0])
new_df=pd.DataFrame(l)
new_df.shape

(145, 5)

In [17]:
new_df.head()

,Locality,District,Pincode,Latitude,Longitude
0,Air Force Hospital,Bangalore,560007,12.964027,77.627500
1,Amruthahalli,Bangalore,560092,13.066513,77.596624
2,Anandnagar Bangalore,Bangalore,560024,13.033377,77.589523
3,Arabic College,Bangalore,560045,13.030013,77.620872
4,Attur,Bangalore,560064,13.107470,77.565685


In [18]:
#Storing final cleaned and required data
new_df.to_csv('Bangalore_cleaned.csv')

In [19]:
#To visualize the final data points
latitude=12.9180502
longitude=77.5922633618582
map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, location in zip(new_df['Latitude'], new_df['Longitude'], new_df['District'], new_df['Locality']):
    label = '{},{}'.format(location,district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bang)  
    
map_bang

<h2>Gathering more data about neighbourhood using Foursquare Api </h2>

In [20]:
#A method To get venues near every location considering 500 metres radius and maximum of 100 venues per location
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
#Storing every venues with the locality
bangalore_venues = getNearbyVenues(names=new_df['Locality'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

Air Force Hospital
Amruthahalli
Anandnagar Bangalore
Arabic College
Attur
Austin Town
Banaswadi
Bangalore International Airport
Bellandur,Ashoknagar Bangalore,Shanthinagar,Bangalore City
Benson Town
Bhattarahalli
Byatarayanapura
C.V.Raman Nagar
CMP Centre & School
Devasandra
Doddanekkundi
Domlur
Dr. Ambedkar Veedhi
EPIP
Hoodi
Horamavu
Indiranagar Bangalore
Jakkur
Kadugodi
Kannamangala
Kothanur
Krishnarajapuram R S,Krishnarajapuram
Kundalahalli
Lingarajapuram
Mahadevapura
Mahatma Gandhi Road
Maruthi Sevanagar
Medimallasandra
Museum Road
NAL
New Thippasandra
Panathur
R T Nagar
Rajanakunte
Ramamurthy Nagar
Richmond Town
Sadashivanagar
Singanayakanahalli
Venkateshapura
Vidhana Soudha
Vimanapura
Virgonagar
Viveknagar Bangalore
Whitefield
Yelahanka
Yelahanka Satellite Town
Adugodi
Anjanapura
Banashankari III Stage
Banashankari
Bangalore Corporation Building
Bannerghatta Road
Bannerghatta
Basavanagudi
Begur
Bolare
Bommasandra Industrial Estate
Chamrajpet Bangalore
Chandapura,Chandapura
Chickp

In [22]:
#total no of venues found
print(bangalore_venues.shape)
bangalore_venues.tail()

(998, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
993,Kannur,13.103799,77.601617,"Udupi Grand, Kogilu Cross",13.103455,77.600627,Fast Food Restaurant
994,Kannur,13.103799,77.601617,Yelahanka lake,13.103729,77.602205,Lake
995,Kannur,13.103799,77.601617,Gobi Adda,13.103732,77.602401,Food Truck
996,Neriga,12.911766,77.776525,Shristhi Village,12.913261,77.773636,Resort
997,Sarjapura,12.860087,77.786135,sarjapur 1. Wipro cafeteria,12.858275,77.783093,Snack Place


<h2>Data Analysis Phase</h2>

In [23]:
bangalore_venues.groupby('Neighborhood').count()
#bangalore_venues[['Venue', 'Venue Category']].corr() 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adugodi,4,4,4,4,4,4
Air Force Hospital,2,2,2,2,2,2
Amruthahalli,4,4,4,4,4,4
Anandnagar Bangalore,5,5,5,5,5,5
Anekal,2,2,2,2,2,2
Anjanapura,1,1,1,1,1,1
Arabic College,1,1,1,1,1,1
Attur,1,1,1,1,1,1
Austin Town,2,2,2,2,2,2


In [24]:
bangalore_venues.to_csv('Bangalore_locations_with_neighbourhood.csv')

In [25]:
bangalore_venues.groupby('Venue Category').count().sort_values(by=['Venue'],ascending=False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Indian Restaurant,163,163,163,163,163,163
Café,71,71,71,71,71,71
Fast Food Restaurant,38,38,38,38,38,38
Coffee Shop,30,30,30,30,30,30
Department Store,28,28,28,28,28,28
Hotel,26,26,26,26,26,26
Bakery,26,26,26,26,26,26
Pizza Place,24,24,24,24,24,24
Chinese Restaurant,23,23,23,23,23,23


<!--trial = bangalore_venues[bangalore_venues['Venue Category']=='Indian Restaurant']
trial-->
<p>ignore<p>

<!--latitude=12.9180502
longitude=77.5922633618582
map_bang = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
<!--for lat, lng, district, location in zip(trial['Venue Latitude'], trial['Venue Longitude'], trial['Venue'], trial['Neighborhood']):
    label = '{},{}'.format(district,location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bang)  
map_bang-->
<p>ignore</p>

<!--trial.groupby('Neighborhood').count()
-->
<p>ignore</p>

In [26]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 176 uniques categories.


<h2>Analyze each neighbourhood</h2>

In [27]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Andhra Restaurant,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Capitol Building,Casino,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Shop,Women's Store,Yoga Studio
0,Air Force Hospital,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Air Force Hospital,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amrut

In [28]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Andhra Restaurant,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Capitol Building,Casino,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.250000,0.00,0.000000,0.250000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Air Force Hospital,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adugodi,Athletics & Sports,Gym,Bus Station,Café,Food & Drink Shop
1,Air Force Hospital,Multiplex,Casino,Yoga Studio,Electronics Store,Food & Drink Shop
2,Amruthahalli,Indian Restaurant,Shoe Store,Print Shop,Convenience Store,Airport
3,Anandnagar Bangalore,Indian Restaurant,Motorcycle Shop,Hotel,Pharmacy,Garden Center
4,Anekal,ATM,Camera Store,Cupcake Shop,Food Truck,Food & Drink Shop


In [40]:
# set number of clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 2, 0, 3, 0, 3])

In [42]:
#neighborhoods_venues_sorted.drop('Cluster Labels',axis=1,inplace=True)

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = new_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Locality')

bangalore_merged.tail() # check the last columns!

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
150,Madanayakanahalli,Bangalore,562162,13.059247,77.461511,NaN,NaN,NaN,NaN,NaN,NaN
151,Mayasandra,Bangalore,562107,12.877244,77.377949,NaN,NaN,NaN,NaN,NaN,NaN
152,Neriga,Bangalore,562125,12.911766,77.776525,3.0,Resort,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food
153,Sarjapura,Bangalore,562125,12.860087,77.786135,4.0,Snack Place,Yoga Studio,Coworking Space,Food & Drink Shop,Food
156,Yadavanahalli,Bangalore,562107,12.789855,77.751454,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
bangalore_merged.tail() 

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
150,Madanayakanahalli,Bangalore,562162,13.059247,77.461511,NaN,NaN,NaN,NaN,NaN,NaN
151,Mayasandra,Bangalore,562107,12.877244,77.377949,NaN,NaN,NaN,NaN,NaN,NaN
152,Neriga,Bangalore,562125,12.911766,77.776525,3.0,Resort,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food
153,Sarjapura,Bangalore,562125,12.860087,77.786135,4.0,Snack Place,Yoga Studio,Coworking Space,Food & Drink Shop,Food
156,Yadavanahalli,Bangalore,562107,12.789855,77.751454,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
bangalore_merged.dropna(inplace=True)
bangalore_merged.reset_index(drop=True,inplace=True)
bangalore_merged.tail()

,Locality,District,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
117,Hunasamaranahalli,Bangalore,562157,13.144010,77.619124,0.0,Indian Restaurant,Coffee Shop,Lake,Eastern European Restaurant,Food & Drink Shop
118,Kadabagere,Bangalore,562130,12.988910,77.449245,0.0,Indian Restaurant,Women's Store,Food Truck,Food & Drink Shop,Food
119,Kannur,Bangalore,562149,13.103799,77.601617,3.0,Food Truck,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Lake,Eastern European Restaurant
120,Neriga,Bangalore,562125,12.911766,77.776525,3.0,Resort,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food
121,Sarjapura,Bangalore,562125,12.860087,77.786135,4.0,Snack Place,Yoga Studio,Coworking Space,Food & Drink Shop,Food


In [46]:
bangalore_merged['Cluster Labels'].unique()

array([3., 0., 1., 2., 4.])

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Locality'],bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2 align=center>Examining Clusters</h2>

<h3>1st Cluster</h3>

In [91]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Byatarayanapura,0.0,Indian Restaurant,Wine Shop,Cosmetics Shop,Andhra Restaurant,Department Store
18,Hoodi,0.0,Indian Restaurant,Yoga Studio,Furniture / Home Store,Dry Cleaner,Flower Shop
32,NAL,0.0,Indian Restaurant,Café,Chinese Restaurant,Dessert Shop,Kerala Restaurant
40,Singanayakanahalli,0.0,Indian Restaurant,Café,Yoga Studio,Food,Flower Shop
68,Gottigere,0.0,Indian Restaurant,Department Store,Dessert Shop,Yoga Studio,Dumpling Restaurant
81,Mavalli,0.0,Indian Restaurant,Hotel,Food Truck,Snack Place,Restaurant
88,Thalaghattapura,0.0,Indian Restaurant,Yoga Studio,Dry Cleaner,Food,Flower Shop
103,Msrit,0.0,Indian Restaurant,Fast Food Restaurant,Bus Station,Yoga Studio,Dumpling Restaurant
118,Hunasamaranahalli,0.0,Indian Restaurant,Coffee Shop,Lake,Donut Shop,Flower Shop
119,Kadabagere,0.0,Indian Restaurant,Yoga Studio,Dry Cleaner,Food,Flower Shop


<p>It has a hub of Indian Restaurants, most of them having some yoga centers and some flower shops</p>

<h3>2nd Cluster</h3>

In [92]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Arabic College,1.0,Coffee Shop,Yoga Studio,Dry Cleaner,Food,Flower Shop
4,Attur,1.0,Juice Bar,Yoga Studio,Dry Cleaner,Food,Flower Shop
8,"Bellandur,Ashoknagar Bangalore,Shanthinagar",1.0,Hotel,Vineyard,Capitol Building,Park,Donut Shop
9,Benson Town,1.0,Candy Store,Café,Dessert Shop,Pakistani Restaurant,Bar
10,Bhattarahalli,1.0,Ice Cream Shop,Hotel,Japanese Restaurant,Café,Dry Cleaner
13,CMP Centre & School,1.0,Hotel,Soccer Field,Café,Sporting Goods Shop,Health Food Store
20,Indiranagar Bangalore,1.0,Lounge,Pub,Café,Ice Cream Shop,Restaurant
22,Kadugodi,1.0,Playground,Platform,Indian Restaurant,Park,Train Station
23,Kothanur,1.0,Pizza Place,Coffee Shop,Bakery,Convenience Store,Gym / Fitness Center
26,Lingarajapuram,1.0,Burger Joint,Andhra Restaurant,Garden Center,Bakery,Café


<p>These are varity of things places with non traditional Indian restaurantsand many fitness places</p>

<h3>3rd Cluster</h3>

In [93]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Air Force Hospital,2.0,Multiplex,Casino,Yoga Studio,Dry Cleaner,Flower Shop
25,Kundalahalli,2.0,Fast Food Restaurant,Hotel,Sports Bar,Dessert Shop,Bakery
29,Maruthi Sevanagar,2.0,Fast Food Restaurant,Clothing Store,Multiplex,Department Store,Shopping Mall
58,Bolare,2.0,Fast Food Restaurant,Yoga Studio,Dry Cleaner,Food,Flower Shop
59,Chamrajpet Bangalore,2.0,Fast Food Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Shop & Service,Yoga Studio
74,Kathriguppe,2.0,Breakfast Spot,Fast Food Restaurant,Smoke Shop,Bus Station,Yoga Studio
84,Nayandahalli,2.0,IT Services,Accessories Store,Metro Station,Fast Food Restaurant,Train Station
115,Chikkajala,2.0,Hotel,Bike Shop,Fast Food Restaurant,Bar,Café
120,Kannur,2.0,Food Truck,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Lake,Dry Cleaner


<p>These places have chain of Fast food Restaurants and multiple entertainment places like mutiplex,casino,sports bar etc</p>

<h3>4th Cluster</h3>

In [94]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Amruthahalli,3.0,Convenience Store,Shoe Store,Indian Restaurant,Department Store,Flower Shop
2,Anandnagar Bangalore,3.0,Hotel,Pharmacy,Flea Market,Garden Center,Motorcycle Shop
5,Austin Town,3.0,Tibetan Restaurant,Indian Restaurant,Italian Restaurant,Bakery,Dry Cleaner
6,Banaswadi,3.0,Vegetarian / Vegan Restaurant,Indian Restaurant,Bakery,BBQ Joint,Yoga Studio
7,Bangalore International Airport,3.0,Coffee Shop,Airport Lounge,Indian Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
12,C.V.Raman Nagar,3.0,Indian Restaurant,Pizza Place,Shop & Service,Park,Department Store
14,Doddanekkundi,3.0,Indian Restaurant,Food,Fried Chicken Joint,Clothing Store,Restaurant
15,Domlur,3.0,Indian Restaurant,Café,Food & Drink Shop,Pub,Sports Bar
16,Dr. Ambedkar Veedhi,3.0,Coffee Shop,Indian Restaurant,Hotel,Capitol Building,Dry Cleaner
17,EPIP,3.0,Indian Restaurant,Ice Cream Shop,Breakfast Spot,Maharashtrian Restaurant,Fried Chicken Joint


<p>These are food hubs of the city with enough restaurants and cafes present</p>

<h3>5th Cluster</h3>

In [95]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Locality,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
21,Jakkur,4.0,ATM,Dry Cleaner,Food,Flower Shop,Flea Market
41,Venkateshapura,4.0,ATM,Indian Restaurant,Motorcycle Shop,Cosmetics Shop,Dumpling Restaurant
63,Deepanjalinagar,4.0,ATM,Metro Station,Travel & Transport,Dry Cleaner,Flower Shop
71,"Hulimavu,Hulimavu",4.0,ATM,Department Store,Lake,Badminton Court,Dumpling Restaurant
78,Kumbalagodu,4.0,ATM,Café,Dhaba,Flower Shop,Flea Market
96,Jalahalli West,4.0,ATM,Dry Cleaner,Food,Flower Shop,Flea Market
100,Magadi Road,4.0,ATM,Photography Studio,Restaurant,Donut Shop,Flea Market
112,Anekal,4.0,ATM,Camera Store,Cosmetics Shop,Dry Cleaner,Flower Shop


<p>These are the locations with majority of ATMs of city and many flea shops</p>

<h2>A notebook by - <i>Sahil Sinha</i></h2>